In [1]:
!pip -q install imbalanced-learn

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek


In [2]:
url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

print("Shape:", df.shape)
df.head()


Shape: (772, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
print("Class distribution (before balancing):")
print(df["Class"].value_counts())


Class distribution (before balancing):
Class
0    763
1      9
Name: count, dtype: int64


In [4]:
X = df.drop("Class", axis=1)
y = df["Class"]


In [5]:
smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)

print("Class distribution (after balancing):")
print(pd.Series(y_bal).value_counts())
print("Balanced shape:", X_bal.shape)


Class distribution (after balancing):
Class
0    763
1    763
Name: count, dtype: int64
Balanced shape: (1526, 30)


In [9]:
samples = []
for i in range(5):
    X_s, _, y_s, _ = train_test_split(X_bal, y_bal, test_size=0.3, random_state=i, stratify=y_bal)
    samples.append((X_s, y_s))

print("Created", len(samples), "samples")
print("Sample 1 shape:", samples[0][0].shape)


Created 5 samples
Sample 1 shape: (1068, 30)


In [10]:
samplers = {
    "Sampling1_RandomUnder": RandomUnderSampler(random_state=42),
    "Sampling2_RandomOver": RandomOverSampler(random_state=42),
    "Sampling3_SMOTE": SMOTE(random_state=42),
    "Sampling4_NearMiss": NearMiss(),
    "Sampling5_SMOTETomek": SMOTETomek(random_state=42)
}

list(samplers.keys())


['Sampling1_RandomUnder',
 'Sampling2_RandomOver',
 'Sampling3_SMOTE',
 'Sampling4_NearMiss',
 'Sampling5_SMOTETomek']

In [11]:
models = {
    "M1_LogisticRegression": LogisticRegression(max_iter=1000),
    "M2_DecisionTree": DecisionTreeClassifier(random_state=42),
    "M3_RandomForest": RandomForestClassifier(n_estimators=200, random_state=42),
    "M4_KNN": KNeighborsClassifier(n_neighbors=5),
    "M5_NaiveBayes": GaussianNB()
}

list(models.keys())


['M1_LogisticRegression',
 'M2_DecisionTree',
 'M3_RandomForest',
 'M4_KNN',
 'M5_NaiveBayes']

In [12]:
results = pd.DataFrame(index=models.keys(), columns=samplers.keys())

for model_name, model in models.items():
    for samp_name, sampler in samplers.items():


        X_s, y_s = samples[0]


        X_res, y_res = sampler.fit_resample(X_s, y_s)


        X_train, X_test, y_train, y_test = train_test_split(
            X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
        )


        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)


        acc = accuracy_score(y_test, y_pred) * 100
        results.loc[model_name, samp_name] = round(acc, 2)

results


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,Sampling1_RandomUnder,Sampling2_RandomOver,Sampling3_SMOTE,Sampling4_NearMiss,Sampling5_SMOTETomek
M1_LogisticRegression,92.06,93.46,93.46,93.46,89.86
M2_DecisionTree,97.66,98.6,98.6,98.13,96.14
M3_RandomForest,100.0,100.0,100.0,100.0,99.03
M4_KNN,82.71,85.98,85.98,83.18,85.99
M5_NaiveBayes,83.18,86.45,86.45,81.78,83.09


In [15]:
best_per_model = results.astype(float).idxmax(axis=1)
best_score_per_model = results.astype(float).max(axis=1)

summary = pd.DataFrame({
    "Best Sampling Technique": best_per_model,
    "Best Accuracy (%)": best_score_per_model.round(2)
})

print("Best sampling technique for each model:")
display(summary)


Best sampling technique for each model:


,Best Sampling Technique,Best Accuracy (%)
M1_LogisticRegression,Sampling2_RandomOver,93.46
M2_DecisionTree,Sampling2_RandomOver,98.60
M3_RandomForest,Sampling1_RandomUnder,100.00
M4_KNN,Sampling5_SMOTETomek,85.99
M5_NaiveBayes,Sampling2_RandomOver,86.45
